In [ ]:
import msprime
import numpy as np
import pandas as pd
import math
from random import shuffle
from primeval import *

### Code for the Chen et al. (2015) model

### Actual function is found in primeval.py file

```python
def chen_model(mu=1.5e-8, phi=0, length=1e4, n_afr=0, n_eur=0, debug=False):

    generation_time = 25
    
    # 220kya:
    # African population constant with Ne~7300
    N_A = 7310
    
    # 148kya:
    # instantaneous growth to Ne~14000
    T_AF = 148e3 / generation_time
    N_AF = 14474
    
    N6_EU = 13143
    
    # 118kya:
    # non-AFR pops migrate OOA; bottlenecks to Ne~1800
    # migration between AFR occurs
    N_B = 1861
    T5 = 118e3 / generation_time
    T4 = T5
    m_AF_B = 15e-5
    N5_EU = 62
    N4_EU = N6_EU
    
    # 18kya:
    # 2nd EUR bottlenecks to Ne~1000 & starts growing with rate 0.307%
    # migration rate slows between AFR-EUR
    N_EU0 = 1032
    T3 = 18e3 / generation_time
    T2 = T3
#     m_AF_EU = 2.5e-5
    r_EU0 = 0 # 0.00307
#     N2_EU = 15829 # N_EU0 / math.exp(-r_EU0 * T_EU_B)
    N2_EU = 16178
    N2_AF = 26682
    N3_EU = 2020
    # 5.1kya:
    # explosive growth in both AFR & EUR
    T1_EU = 4.95e3 / generation_time
    T1_AF = 10.01e3 / generation_time
    r_EU = 0.022
    r_AF = 0.00735
    N1_EU = 1.261e6
    m_EG = 0
    N1_AF = 5.062e5
    
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=n_afr, initial_size=N1_AF, growth_rate=r_AF),
        msprime.PopulationConfiguration(
            sample_size=n_eur, initial_size=N1_EU, growth_rate=r_EU)#,
    ]

    # up to 5.1kya, no migration
    migration_matrix = [
        [0, 0],
        [0, 0],
    ]
    
    demographic_events = [
        msprime.PopulationParametersChange(
            time=T1_EU, growth_rate=0, initial_size=N2_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T1_AF, growth_rate=0, initial_size=N2_AF, population_id=0),
        
        # at 18kya, bottleneck + instantaneous recovery to smaller Ne
        msprime.PopulationParametersChange(
            time=T2, initial_size=N3_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T3, initial_size=N4_EU, population_id=1),
        
        # at 118kya, bottleneck + instantaneous recovery to same Ne
        msprime.PopulationParametersChange(
            time=T4, initial_size=N5_EU, population_id=1),
        msprime.PopulationParametersChange(
            time=T5, initial_size=N6_EU, population_id=1),
        
        # At 148kya, instantaneous growth in AFR
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    
    if(debug):
        # Use the demography debugger to print out the demographic history
        # that we have just described.
        dd = msprime.DemographyDebugger(
            population_configurations=population_configurations,
            migration_matrix=migration_matrix,
            demographic_events=demographic_events)
        dd.print_history()
    else:
        sim = msprime.simulate(population_configurations=population_configurations,
                               migration_matrix=migration_matrix, 
                               demographic_events=demographic_events,
                               mutation_rate=mu, 
                               recombination_rate=phi, 
                               length=length,
                               random_seed=30)
        return sim
```

### Run with debugging to check correct parameters at each epoch

In [ ]:
chen_model(debug=True)